In [29]:
__author__ = "Vikas Chitturi"
__tags__   = ['elastic_search', 'pids','json','parsing']

In [30]:
from elasticsearch import Elasticsearch
import pandas as pd
import json
from pprint import pprint
from collections import defaultdict

In [37]:
#test the elastic search api endpoint
es_conn = Elasticsearch([{'host':'es1.hs.internal','port':9200}],timeout=10)
es_conn.ping()

True

In [38]:
es_conn.indices.get_alias("*")

{'hsboutique': {'aliases': {}},
 'hsproductcontentsp': {'aliases': {}},
 'hscollectionsimilarproducts': {'aliases': {}},
 'hsproductattribute': {'aliases': {}},
 'hs': {'aliases': {}},
 'hscollectionprofile': {'aliases': {}},
 'hsproductnew': {'aliases': {}},
 'hscollabfiltering': {'aliases': {}},
 'hscustomproduct': {'aliases': {}},
 'hscustomtiles': {'aliases': {}},
 '.elastichq': {'aliases': {}},
 'hssimilar_collection': {'aliases': {}},
 'hssimilarproducts': {'aliases': {}},
 'hscustomerprofile': {'aliases': {}},
 'hscategorygrouping': {'aliases': {}},
 'hspersonalisation': {'aliases': {}}}

In [42]:
test_body = es_conn.search(index='hsproductnew',body={'from':0,'size':200,'query':{}})

In [46]:
total_num = test_body['hits']['total']
total_num

141804

In [43]:
type(test_body)

dict

In [44]:
len(test_body['hits']['hits'])

200

In [49]:
result_set = defaultdict(list)
for pid_num in range(0,total_num,1000):
    data = es_conn.search(index='hsproductnew',body={'from':pid_num,'size':1000,'query':{}})
    #print (len(data['hits']['hits']))
    for i in data['hits']['hits']:
        mykeys = list(i['_source'].keys())
        url_index = mykeys.index('imgurls')
        skus_index = mykeys.index('skus')
        required_keys = mykeys[url_index+1:skus_index]
        required_keys.append('product_id')
        [result_set[col].append(i['_source'][col]) for col in required_keys]
    if pid_num%10000 == 0:
        print (pid_num)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [53]:
result_set.keys()

dict_keys(['image_count', 'stock_type', 'product_add_to_site_date', 'brand_id', 'Secondary Color', 'Season', 'Colour', 'Closure_Footwear', 'Year', 'Toe Style', 'Occasion', 'Shaft_Footwear', 'HBT', 'Gender', 'Accents_Footwear', 'Pattern_Footwear', 'product_id', 'Heel Style', 'Sleeves', 'Neckline', 'Bottom Length', 'Stitch', 'Top Closure', 'Bottom Pattern', 'Top Pattern', 'Units per sets', 'Top Length', 'Taste', 'Bottom Closure', 'Top Print', 'Top Style', 'Bottom Print', 'Print', 'Closure', 'Length', 'Style', 'Pattern', 'Outsoles', 'Upper Materials', 'Insoles', 'Accents', 'Collar', 'Bottom Style', 'Split Shipped', 'Type', 'Heel Length', 'Warning', 'Features', 'Care', "What's included", 'Fabric', 'Material', 'Book Details', 'Important Note', 'Product Length', 'Product Width', 'UPS', 'Warranty', 'Automobile', 'On Sale', 'Animal'])

In [55]:
result_set = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in result_set.items() ]))

In [71]:
result_set.shape

(141804, 61)

In [73]:
result_set.head(15)

,image_count,stock_type,product_add_to_site_date,brand_id,Secondary Color,Season,Colour,Closure_Footwear,Year,Toe Style,...,Material,Book Details,Important Note,Product Length,Product Width,UPS,Warranty,Automobile,On Sale,Animal
0,2,Out Of Stock,2019-04-10T18:30:00Z,13296,[Pink],[All Season],[Pink],"[Fixed Laces, Velcro]",2019,Round,...,Leather,"6,Board books,English,Julia Donaldson",All of our products are lead free and thorough...,147,65,5037970006157,5 Year,,[Y],[Elephant]
1,4,In Stock,2019-04-14T18:30:00Z,12836,"[Shell, Strom]",[All Season],[Green],[Velcro],2019,Open,...,Leather,"32,English,Ian Graham,Paperback,Silver Dolphin...",Age Range: 8 and up,1,1,026602270122,"World Kitchen will repair or replace, or at Wo...",,[Y],[Elephant]
2,3,Out Of Stock,2019-07-22T18:30:00Z,15877,[Frostlilac],[All Season],[White],"[Fixed Laces, Velcro]",2019,Round,...,Leather,"32,Dennis Schatz,English,Paperback,Silver Dolp...",Age Range: 6 and up,1,1,026602264640,5 Year,,[Y],[Dog]
3,4,Broken Size,2019-03-07T18:30:00Z,14483,[Off White],[All Season],"[Shell, Strom]","[Fixed Laces, Velcro]",2019,Round,...,Paper,"32,English,Paperback,Ruth Strother,Silver Dolp...",Age Range: 0 to 3,1,1,8901736096713,6 months Manufacturer Warranty,,[Y],[Bug]
4,8,Fat Size,2019-04-17T18:30:00Z,13521,[Strom],[All Season],[Aqua],[Velcro],2019,Open,...,Paper,"14,English,Paperback,Silver Dolphin Books,Sue ...",We would also recommend our JoJo sunglasses wh...,1,1,8901736097512,One-year manufacturer's warranty against defec...,,[Y],[Giraffe]
5,3,In Stock,2019-09-14T18:30:00Z,14899,"[Black, Off White]",[Winter],[Black],[Velcro],2019,Capped,...,Other,"16,English,Ladybird,Paperback,Peppa Pig",This is an non-returnable product. Please use ...,1,1,026602143655,6 months Manufacturer Warranty,,[N],[Butterfly]
6,3,In Stock,2019-09-02T18:30:00Z,12337,[Silver],[Summer],[Frostlilac],"[Laces, Velcro]",2019,Round,...,Other,"56,English,Hardcover,Ladybird(Penguin UK),Pete...",Best suited for Women: -Full / Semi full breas...,1,1,8907182010907,one year manufacturer's warranty,,[N],[Butterfly]
7,6,In Stock,2019-10-29T18:30:00Z,15987,[Black],[Winter],[Off White],[Velcro],2019,Open,...,,"Ladybird(Penguin UK),56,English,Hardcover,Pete...",Best suited for Women: -Full / Semi full breas...,1,1,871853007108,One-year manufacturer's warranty against defec...,,[N],[Monkey]
8,3,Out Of Stock,2019-04-15T18:30:00Z,13296,"[Black, Silver]",[All Season],[Navy],"[Laces, Velcro]",2019,Round,...,Other,"56,English,Hardcover,Ladybird(Penguin UK),Pete...",Best suited for Women: -Full / Semi full breas...,1,1,871853005579,One-year manufacturer's warranty against defec...,,[N],[Giraffe]
9,1,In Stock,2018-07-16T18:30:00Z,15223,"[Brown, Flame, Tan]",[All Season],[Maroon],[Velcro],2018,Round,...,Other,"56,English,Hardcover,Ladybird(Penguin UK),Pete...",Best suited for Women: -Full / Semi full breas...,246,242,8901736097291,One-year manufacturer's warranty against defec...,,[N],[Bug]


In [61]:
print (result_set['product_id'].head(10))
result_set.dtypes

0    807749
1    810735
2    855282
3    792578
4    804578
5    868009
6    862765
7    887782
8    810346
9    685691
Name: product_id, dtype: object


image_count                  int64
stock_type                  object
product_add_to_site_date    object
brand_id                    object
Secondary Color             object
Season                      object
Colour                      object
Closure_Footwear            object
Year                        object
Toe Style                   object
Occasion                    object
Shaft_Footwear              object
HBT                         object
Gender                      object
Accents_Footwear            object
Pattern_Footwear            object
product_id                  object
Heel Style                  object
Sleeves                     object
Neckline                    object
Bottom Length               object
Stitch                      object
Top Closure                 object
Bottom Pattern              object
Top Pattern                 object
Units per sets              object
Top Length                  object
Taste                       object
Bottom Closure      

In [70]:
result_set[result_set['product_id']=='574192']

,image_count,stock_type,product_add_to_site_date,brand_id,Secondary Color,Season,Colour,Closure_Footwear,Year,Toe Style,...,Material,Book Details,Important Note,Product Length,Product Width,UPS,Warranty,Automobile,On Sale,Animal
2177,3,Broken Size,2017-10-17T18:30:00Z,14420,[Shell],[All Season],[Clouds],[Elastic],2018,Round,...,Other,NaN,NaN,,,NaN,NaN,,NaN,NaN
5878,3,Broken Size,2017-10-17T18:30:00Z,14420,"[Olive, Orange, Tan]",[All Season],[Clouds],[Velcro],2019,Round,...,,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN
6846,3,Broken Size,2017-10-17T18:30:00Z,14420,[Strom],[All Season],[Clouds],[Laces],2019,Round,...,Other,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN
141502,3,Broken Size,2017-10-17T18:30:00Z,14420,NaN,NaN,[Blue],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
result_set['product_id'].nunique()

102285

In [66]:
result_set['product_id'].value_counts()

574192    4
541511    4
21388     4
551369    4
69136     4
547995    4
543325    4
136901    4
853785    4
21735     4
67427     3
55245     3
686162    3
885528    3
861320    3
848378    3
751340    3
551750    3
878603    3
19481     3
27381     3
21583     3
20856     3
543060    3
782528    3
339241    3
849634    3
103865    3
858377    3
618367    3
         ..
191230    1
667098    1
90985     1
862803    1
116876    1
798476    1
700778    1
98080     1
575850    1
619175    1
800235    1
81192     1
99331     1
844781    1
694548    1
519745    1
56883     1
860085    1
77988     1
858671    1
889275    1
49450     1
110458    1
129118    1
903291    1
883144    1
877217    1
874605    1
797039    1
739112    1
Name: product_id, Length: 102285, dtype: int64

In [69]:
result_set.isnull().sum()

image_count                      0
stock_type                       0
product_add_to_site_date         2
brand_id                         0
Secondary Color             107220
Season                         695
Colour                           1
Closure_Footwear            133442
Year                           803
Toe Style                   130539
Occasion                    110590
Shaft_Footwear              138995
HBT                          41138
Gender                           1
Accents_Footwear            135348
Pattern_Footwear            135498
product_id                       0
Heel Style                  136746
Sleeves                     113936
Neckline                    127617
Bottom Length               134478
Stitch                      140548
Top Closure                 138480
Bottom Pattern              137250
Top Pattern                 138705
Units per sets              133997
Top Length                  134073
Taste                       119202
Bottom Closure      

In [67]:
result_set.to_csv('parsed_product_events.csv',index=False)